# Caso de uso para la aplicación

In [1]:
import pymongo
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, POST, JSON
import requests
import pprint
import re

d:\Proyectos\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_config = dotenv_values("../.env")

### Aux functions

In [3]:
def to_title_case(s):
    return re.sub(r"(\s+|[;,.])", "", s.title())

def to_camel_case(s):
    title_case = to_title_case(s)
    return title_case[0].lower() + title_case[1:]

def get_titulos(boe_ids):
    sparql.setQuery(PREFIXES + f"""
    SELECT ?entradaBOE ?titulo
    WHERE {{
    ?entradaBOE rdf:type :EntradaBOE .
    ?entradaBOE :identificador "{boe_id}" .
    ?entradaBOE :titulo ?titulo .
    }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"][0]

### Creando conexiones

In [4]:
mongo_client = pymongo.MongoClient(
    host=env_config['MONGODB_HOST'],
    port=int(env_config['MONGODB_PORT']),
    username=env_config['MONGO_USER'],
    password=env_config['MONGO_PASSWORD'],
)
mongo_collection = mongo_client["boe_db"]["boe"]

In [5]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

In [6]:
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}"
)
sparql.setMethod(POST)

## Selección del BOE(s) que aplican al tema elegido

In [7]:
tema_elegido = "(1E-2023) para la provisión de puestos de trabajo"

### Búsqueda semántica

In [10]:
query_text = "cantidad de agua tope"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint.pprint(res["hits"])

{'hits': [], 'max_score': None, 'total': {'relation': 'eq', 'value': 0}}


## Tratamiento del BOE objetivo en GraphDB

In [ ]:
boe_id = "BOE-A-2023-8320"

### Mapeando posibles relaciones

In [ ]:
posteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"posteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["posteriores"]:
        relacion = anterior["relacion"]
        posteriores[relacion["codigo"]] = to_camel_case(relacion["texto"])

anteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"anteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["anteriores"]:
        relacion = anterior["relacion"]
        anteriores[relacion["codigo"]] = to_camel_case(relacion["texto"])

In [ ]:
v_relaciones = dict()
v_relaciones['anteriores'] = list(anteriores.values())
v_relaciones['posteriores'] = list(posteriores.values())

### Consulta documento objetivo

In [ ]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

#### Procesamos el documento elegido y todos sus atributos y relaciones

In [ ]:
describe_query = PREFIXES + f"""
                DESCRIBE ?entradaBOE
                WHERE {{
                ?entradaBOE rdf:type :EntradaBOE .
                ?entradaBOE :identificador "{boe_id}" .
                }}
                """
sparql.setQuery(query=describe_query)
sparql.setReturnFormat(JSON)
sparql.addParameter('Content-Type', 'application/sparql-query')
results = sparql.query().convert().decode("utf-8")

In [ ]:
found_rels = dict()
for relacion in v_relaciones['anteriores']:
    if relacion in results:
        found_rels[relacion] = 'anterior'

for relacion in v_relaciones['posteriores']:
    if relacion in results:
        found_rels[relacion] = 'posterior'

#### Obtención de BOEs relacionados

In [ ]:
def get_referenced_boes(boe_id, relaciones):
    select = "SELECT ?entradaBOE " + " ".join([f"?boe_anterior_{relacion} ?boe_posterior_{relacion} " for relacion in relaciones.keys()])
    where = f"""
            WHERE {{
                ?entradaBOE rdf:type :EntradaBOE .
                ?entradaBOE :identificador "{boe_id}" .
            """
    for relacion in relaciones.keys():
        if relaciones[relacion] == 'anterior':
            get_relacion_ant = f"""
                OPTIONAL {{
                        ?entradaBOE :{relacion} ?boe_anterior_{relacion} .
                        FILTER EXISTS {{ ?entradaBOE :{relacion} ?boe_anterior_{relacion} }}
                    }}
            """
            where += get_relacion_ant
        else:
            get_relacion_post = f"""
                OPTIONAL {{
                            ?entradaBOE :{relacion} ?boe_posterior_{relacion} .
                            FILTER EXISTS {{ ?entradaBOE :{relacion} ?boe_posterior_{relacion} }}
                        }}
            """
            where += get_relacion_post
    where += "\n}"

    query_relaciones = PREFIXES + select + where
    sparql.setQuery(query_relaciones)
    sparql.setReturnFormat(JSON)
    sparql.addParameter('Content-Type', 'application/sparql-query')
    results = sparql.query().convert()
    
    anteriores = []
    posteriores = []
    headers = results["head"]["vars"]
    for result in results["results"]["bindings"]:
        entradaBOE = result["entradaBOE"]["value"].split("/")[-1]
        for header in headers:
            try:    
                if result[header]["value"].split("/")[-1] == boe_id: continue
            except:
                continue
            if header.startswith("boe_anterior"):
                anteriores.append((result[header]["value"].split("/")[-1], header.split("_")[-1]))
            else:
                posteriores.append((result[header]["value"].split("/")[-1], header.split("_")[-1]))
    
    return {'anteriores': anteriores, 'posteriores': posteriores}

#### Muestra de los BOEs relacionados

In [ ]:
refs = get_referenced_boes(boe_id, found_rels)
print(f"Entrada seleccionada del BOE: {boe_id}")

print("\nBOEs anteriores:")
for ref in refs['anteriores']:
    print(f"\t*{ref[1]} de {ref[0]}: {get_titulos(ref[0])['titulo']['value']}")

print("\nBOEs posteriores:")
for ref in refs['posteriores']:
    print(f"\t*{ref[1]} en {ref[0]}: {get_titulos(ref[0])['titulo']['value']}")